In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from aequitas.audit import Audit
from aequitas.fairness import Fairness
import aequitas.plot as ap
import experiment_util as util
import models

# [Audit Model's Prediction](https://colab.research.google.com/github/dssg/aequitas/blob/notebooks/compas_demo.ipynb)

In [3]:
raw_df=pd.read_csv("../Datasets/Research/COMPAS/compas-scores-two-years.csv",index_col=0)
df=raw_df.copy()
df

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,0,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,0,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,0,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0


In [4]:
pre_df=util.preporcess_transform(df)
train,test=train_test_split(pre_df,test_size=0.25)

In [5]:
label,score=models.RandomForest(train,test,"two_year_recid")

In [6]:
pred=pd.Series(score,index=label.index,name="pred")
result=pd.concat([df.loc[label.index,["race","sex","age_cat"]],label,pred],axis=1)
result.columns=["race","sex","age_cat","label_value","score"]
result

,race,sex,age_cat,label_value,score
id,,,,,
2467,African-American,Male,25 - 45,0.0,0.0
1608,Caucasian,Female,Greater than 45,1.0,1.0
4387,African-American,Male,25 - 45,1.0,1.0
9730,Caucasian,Male,25 - 45,0.0,0.0
10184,African-American,Male,25 - 45,0.0,0.0
...,...,...,...,...,...
280,Caucasian,Female,25 - 45,1.0,1.0
8390,Caucasian,Male,25 - 45,0.0,0.0
9231,Caucasian,Male,25 - 45,0.0,0.0


In [8]:
audit = Audit(result, label_column="label_value",sensitive_attribute_column=["race","sex","age_cat"])
audit.audit()

In [9]:
audit.confusion_matrix

pp   pn  fp  fn   tn   tp  group_label_pos  \
attribute_name attribute_value                                                 
race           African-American  488  409  27   0  409  461              461   
               Asian               4    5   0   0    5    4                4   
               Caucasian         268  351  16   0  351  252              252   
               Hispanic           54  108   4   0  108   50               50   
               Native American     3    4   1   0    4    2                2   
               Other              43   67   1   0   67   42               42   
sex            Female            134  220   9   0  220  125              125   
               Male              726  724  40   0  724  686              686   
age_cat        25 - 45           499  522  34   0  522  465              465   
               Greater than 45   138  267   5   0  267  133              133   
               Less than 25      223  155  10   0  155  213              213   

                                 group_label_neg  group_size  total_entities  
attribute_name attribute_value                                                
race           African-American              436         897            1804  
               Asian                           5           9            1804  
               Caucasian                     367         619            1804  
               Hispanic                      112         162            1804  
               Native American                 5           7            1804  
               Other                          68         110            1804  
sex            Female                        229         354            1804  
               Male                          764        1450            1804  
age_cat        25 - 45                       556        1021            1804  
               Greater than 45               272         405            1804  
               Less than 25                  165         378            1804

In [10]:
audit.metrics.round(5)

accuracy  tpr      tnr  for      fdr  \
attribute_name attribute_value                                          
race           African-American   0.96990  1.0  0.93807  0.0  0.05533   
               Asian              1.00000  1.0  1.00000  0.0  0.00000   
               Caucasian          0.97415  1.0  0.95640  0.0  0.05970   
               Hispanic           0.97531  1.0  0.96429  0.0  0.07407   
               Native American    0.85714  1.0  0.80000  0.0  0.33333   
               Other              0.99091  1.0  0.98529  0.0  0.02326   
sex            Female             0.97458  1.0  0.96070  0.0  0.06716   
               Male               0.97241  1.0  0.94764  0.0  0.05510   
age_cat        25 - 45            0.96670  1.0  0.93885  0.0  0.06814   
               Greater than 45    0.98765  1.0  0.98162  0.0  0.03623   
               Less than 25       0.97354  1.0  0.93939  0.0  0.04484   

                                     fpr  fnr  npv  precision      ppr  \
attribute_name attribute_value                                           
race           African-American  0.06193  0.0  1.0    0.94467  0.56744   
               Asian             0.00000  0.0  1.0    1.00000  0.00465   
               Caucasian         0.04360  0.0  1.0    0.94030  0.31163   
               Hispanic          0.03571  0.0  1.0    0.92593  0.06279   
               Native American   0.20000  0.0  1.0    0.66667  0.00349   
               Other             0.01471  0.0  1.0    0.97674  0.05000   
sex            Female            0.03930  0.0  1.0    0.93284  0.15581   
               Male              0.05236  0.0  1.0    0.94490  0.84419   
age_cat        25 - 45           0.06115  0.0  1.0    0.93186  0.58023   
               Greater than 45   0.01838  0.0  1.0    0.96377  0.16047   
               Less than 25      0.06061  0.0  1.0    0.95516  0.25930   

                                   pprev     prev  
attribute_name attribute_value                     
race           African-American  0.54404  0.51394  
               Asian             0.44444  0.44444  
               Caucasian         0.43296  0.40711  
               Hispanic          0.33333  0.30864  
               Native American   0.42857  0.28571  
               Other             0.39091  0.38182  
sex            Female            0.37853  0.35311  
               Male              0.50069  0.47310  
age_cat        25 - 45           0.48874  0.45544  
               Greater than 45   0.34074  0.32840  
               Less than 25      0.58995  0.56349

In [11]:
xtab=audit.group.get_crosstabs(result, label_col="label_value",attr_cols=["race","sex","age_cat"])[0]
xtab

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,860,race,African-American,0.969900,1.0,0.938073,0.0,0.055328,...,0.544036,27,0,409,461,461,436,897,1804,0.513935
1,0,binary 0/1,860,race,Asian,1.000000,1.0,1.000000,0.0,0.000000,...,0.444444,0,0,5,4,4,5,9,1804,0.444444
2,0,binary 0/1,860,race,Caucasian,0.974152,1.0,0.956403,0.0,0.059701,...,0.432956,16,0,351,252,252,367,619,1804,0.407108
3,0,binary 0/1,860,race,Hispanic,0.975309,1.0,0.964286,0.0,0.074074,...,0.333333,4,0,108,50,50,112,162,1804,0.308642
4,0,binary 0/1,860,race,Native American,0.857143,1.0,0.800000,0.0,0.333333,...,0.428571,1,0,4,2,2,5,7,1804,0.285714
5,0,binary 0/1,860,race,Other,0.990909,1.0,0.985294,0.0,0.023256,...,0.390909,1,0,67,42,42,68,110,1804,0.381818
6,0,binary 0/1,860,sex,Female,0.974576,1.0,0.960699,0.0,0.067164,...,0.378531,9,0,220,125,125,229,354,1804,0.353107
7,0,binary 0/1,860,sex,Male,0.972414,1.0,0.947644,0.0,0.055096,...,0.500690,40,0,724,686,686,764,1450,1804,0.473103
8,0,binary 0/1,860,age_cat,25 - 45,0.966699,1.0,0.938849,0.0,0.068136,...,0.488737,34,0,522,465,465,556,1021,1804,0.455436
9,0,binary 0/1,860,age_cat,Greater than 45,0.987654,1.0,0.981618,0.0,0.036232,...,0.340741,5,0,267,133,133,272,405,1804,0.328395


In [12]:
audit = Audit(result,label_column="label_value",sensitive_attribute_column=["race","sex","age_cat"],
              reference_groups={'race':'Caucasian', 'sex':'Male', 'age_cat':'25 - 45'})

audit.audit(bias_args={
    "alpha": 0.05,
    "check_significance": True,
    "mask_significance": True,

})

audit.disparity_df.style

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3310: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3310: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: divide by

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,label_value_significance,score_significance,fdr_disparity,fdr_ref_group_value,fdr_significance,fnr_disparity,fnr_ref_group_value,fnr_significance,for_disparity,for_ref_group_value,for_significance,fpr_disparity,fpr_ref_group_value,fpr_significance,npv_disparity,npv_ref_group_value,npv_significance,ppr_disparity,ppr_ref_group_value,ppr_significance,pprev_disparity,pprev_ref_group_value,pprev_significance,precision_disparity,precision_ref_group_value,precision_significance,tnr_disparity,tnr_ref_group_value,tnr_significance,tpr_disparity,tpr_ref_group_value,tpr_significance
0,0,binary 0/1,860,race,African-American,0.969900,1.000000,0.938073,0.000000,0.055328,0.061927,0.000000,1.000000,0.944672,488,409,0.567442,0.544036,27,0,409,461,461,436,897,1804,0.513935,True,True,0.926742,Caucasian,False,nan,Caucasian,None,nan,Caucasian,None,1.420442,Caucasian,False,1.000000,Caucasian,None,1.820896,Caucasian,True,1.256560,Caucasian,True,1.004651,Caucasian,False,0.980835,Caucasian,False,1.000000,Caucasian,None
1,0,binary 0/1,860,race,Asian,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,4,5,0.004651,0.444444,0,0,5,4,4,5,9,1804,0.444444,False,False,0.000000,Caucasian,False,nan,Caucasian,None,nan,Caucasian,None,0.000000,Caucasian,False,1.000000,Caucasian,None,0.014925,Caucasian,False,1.026534,Caucasian,False,1.063492,Caucasian,False,1.045584,Caucasian,False,1.000000,Caucasian,None
2,0,binary 0/1,860,race,Caucasian,0.974152,1.000000,0.956403,0.000000,0.059701,0.043597,0.000000,1.000000,0.940299,268,351,0.311628,0.432956,16,0,351,252,252,367,619,1804,0.407108,False,False,1.000000,Caucasian,False,nan,Caucasian,None,nan,Caucasian,None,1.000000,Caucasian,False,1.000000,Caucasian,None,1.000000,Caucasian,False,1.000000,Caucasian,False,1.000000,Caucasian,False,1.000000,Caucasian,False,1.000000,Caucasian,None
3,0,binary 0/1,860,race,Hispanic,0.975309,1.000000,0.964286,0.000000,0.074074,0.035714,0.000000,1.000000,0.925926,54,108,0.062791,0.333333,4,0,108,50,50,112,162,1804,0.308642,True,True,1.240741,Caucasian,False,nan,Caucasian,None,nan,Caucasian,None,0.819196,Caucasian,False,1.000000,Caucasian,None,0.201493,Caucasian,True,0.769900,Caucasian,True,0.984715,Caucasian,False,1.008242,Caucasian,False,1.000000,Caucasian,None
4,0,binary 0/1,860,race,Native American,0.857143,1.000000,0.800000,0.000000,0.333333,0.200000,0.000000,1.000000,0.666667,3,4,0.003488,0.428571,1,0,4,2,2,5,7,1804,0.285714,False,False,5.583333,Caucasian,False,nan,Caucasian,None,nan,Caucasian,None,4.587500,Caucasian,False,1.000000,Caucasian,None,0.011194,Caucasian,False,0.989872,Caucasian,False,0.708995,Caucasian,False,0.836467,Caucasian,False,1.000000,Caucasian,None
5,0,binary 0/1,860,race,Other,0.990909,1.000000,0.985294,0.000000,0.023256,0.014706,0.000000,1.000000,0.976744,43,67,0.050000,0.390909,1,0,67,42,42,68,110,1804,0.381818,False,False,0.389535,Caucasian,False,nan,Caucasian,None,nan,Caucasian,None,0.337316,Caucasian,False,1.000000,Caucasian,None,0.160448,Caucasian,False,0.902883,Caucasian,False,1.038760,Caucasian,False,1.030208,Caucasian,False,1.000000,Caucasian,None
6,0,binary 0/1,860,sex,Female,0.974576,1.000000,0.960699,0.000000,0.067164,0.039301,0.000000,1.000000,0.932836,134,220,0.155814,0.378531,9,0,220,125,125,229,354,1804,0.353107,True,True,1.219030,Male,False,nan,Male,None,nan,Male,None,0.750655,Male,False,1.000000,Male,None,0.184573,Male,True,0.756019,Male,True,0.987229,Male,False,1.013776,Male,False,1.000000,Male,None
7,0,binary 0/1,860,sex,Male,0.972414,1.000000,0.947644,0.000000,0.055096,0.052356,0.000000,1.000000,0.944904,726,724,0.844186,0.500690,40,0,724,686,686,764,1450,1804,0.473103,False,False,1.000000,Male,False,nan,Male,None,nan,Male,None,1.000000,Male,False,1.000000,Male,None,1.000000,Male,False,1.000000,Male,False,1.000000,Male,False

In [13]:
audit.disparities.style

In [14]:
metrics = ['fpr','fdr']
disparity_tolerance = 1.25

In [15]:
audit.summary_plot(metrics=metrics, fairness_threshold=disparity_tolerance)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To

alt.VConcatChart(...)

In [16]:
audit.disparity_plot(metrics=metrics, attribute='race', fairness_threshold=disparity_tolerance)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\

alt.LayerChart(...)

In [17]:
bdf=audit.disparity_df

In [18]:
ap.absolute(bdf, metrics, 'race', fairness_threshold = disparity_tolerance)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\

alt.LayerChart(...)

In [19]:
ap.disparity(bdf, metrics, 'sex', fairness_threshold = disparity_tolerance)

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\vegalite\v5\api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\altair\

alt.LayerChart(...)

In [20]:
hbdf = audit.bias.get_disparity_predefined_groups(xtab, original_df=result,
                                         ref_groups_dict={'race':'Hispanic', 'sex':'Male', 'age_cat':'25 - 45'},
                                         alpha=0.05,
                                         check_significance=True,
                                         mask_significance=True,
                                         selected_significance=['fpr', 'for', 'fdr'])
hbdf

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,ppr_disparity,ppr_ref_group_value,pprev_disparity,pprev_ref_group_value,precision_disparity,precision_ref_group_value,tnr_disparity,tnr_ref_group_value,tpr_disparity,tpr_ref_group_value
0,0,binary 0/1,860,race,African-American,0.969900,1.0,0.938073,0.0,0.055328,...,9.037037,Hispanic,1.632107,Hispanic,1.020246,Hispanic,0.972817,Hispanic,1.0,Hispanic
1,0,binary 0/1,860,race,Asian,1.000000,1.0,1.000000,0.0,0.000000,...,0.074074,Hispanic,1.333333,Hispanic,1.080000,Hispanic,1.037037,Hispanic,1.0,Hispanic
2,0,binary 0/1,860,race,Caucasian,0.974152,1.0,0.956403,0.0,0.059701,...,4.962963,Hispanic,1.298869,Hispanic,1.015522,Hispanic,0.991826,Hispanic,1.0,Hispanic
3,0,binary 0/1,860,race,Hispanic,0.975309,1.0,0.964286,0.0,0.074074,...,1.000000,Hispanic,1.000000,Hispanic,1.000000,Hispanic,1.000000,Hispanic,1.0,Hispanic
4,0,binary 0/1,860,race,Native American,0.857143,1.0,0.800000,0.0,0.333333,...,0.055556,Hispanic,1.285714,Hispanic,0.720000,Hispanic,0.829630,Hispanic,1.0,Hispanic
5,0,binary 0/1,860,race,Other,0.990909,1.0,0.985294,0.0,0.023256,...,0.796296,Hispanic,1.172727,Hispanic,1.054884,Hispanic,1.021786,Hispanic,1.0,Hispanic
6,0,binary 0/1,860,sex,Female,0.974576,1.0,0.960699,0.0,0.067164,...,0.184573,Male,0.756019,Male,0.987229,Male,1.013776,Male,1.0,Male
7,0,binary 0/1,860,sex,Male,0.972414,1.0,0.947644,0.0,0.055096,...,1.000000,Male,1.000000,Male,1.000000,Male,1.000000,Male,1.0,Male
8,0,binary 0/1,860,age_cat,25 - 45,0.966699,1.0,0.938849,0.0,0.068136,...,1.000000,25 - 45,1.000000,25 - 45,1.000000,25 - 45,1.000000,25 - 45,1.0,25 - 45
9,0,binary 0/1,860,age_cat,Greater than 45,0.987654,1.0,0.981618,0.0,0.036232,...,0.276553,25 - 45,0.697187,25 - 45,1.034237,25 - 45,1.045554,25 - 45,1.0,25 - 45


In [21]:
hbdf[['attribute_name', 'attribute_value'] +
     audit.bias.list_disparities(hbdf) + audit.bias.list_significance(hbdf)]

,attribute_name,attribute_value,fdr_disparity,fnr_disparity,for_disparity,fpr_disparity,npv_disparity,ppr_disparity,pprev_disparity,precision_disparity,tnr_disparity,tpr_disparity,label_value_significance,score_significance,fdr_significance,for_significance,fpr_significance
0,race,African-American,0.746926,NaN,NaN,1.733945,1.0,9.037037,1.632107,1.020246,0.972817,1.0,True,True,False,None,False
1,race,Asian,0.000000,NaN,NaN,0.000000,1.0,0.074074,1.333333,1.080000,1.037037,1.0,False,False,False,None,False
2,race,Caucasian,0.805970,NaN,NaN,1.220708,1.0,4.962963,1.298869,1.015522,0.991826,1.0,True,True,False,None,False
3,race,Hispanic,1.000000,NaN,NaN,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,False,False,False,None,False
4,race,Native American,4.500000,NaN,NaN,5.600000,1.0,0.055556,1.285714,0.720000,0.829630,1.0,False,False,False,None,False
5,race,Other,0.313953,NaN,NaN,0.411765,1.0,0.796296,1.172727,1.054884,1.021786,1.0,False,False,False,None,False
6,sex,Female,1.219030,NaN,NaN,0.750655,1.0,0.184573,0.756019,0.987229,1.013776,1.0,True,True,False,None,False
7,sex,Male,1.000000,NaN,NaN,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,False,False,False,None,False
8,age_cat,25 - 45,1.000000,NaN,NaN,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,False,False,False,None,False
9,age_cat,Greater than 45,0.531756,NaN,NaN,0.300606,1.0,0.276553,0.697187,1.034237,1.045554,1.0,True,True,False,None,False


In [23]:
majority_bdf = audit.bias.get_disparity_major_group(xtab, original_df=df)
majority_bdf

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,860,race,African-American,0.969900,1.0,0.938073,0.0,0.055328,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
1,0,binary 0/1,860,race,Asian,1.000000,1.0,1.000000,0.0,0.000000,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
2,0,binary 0/1,860,race,Caucasian,0.974152,1.0,0.956403,0.0,0.059701,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
3,0,binary 0/1,860,race,Hispanic,0.975309,1.0,0.964286,0.0,0.074074,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
4,0,binary 0/1,860,race,Native American,0.857143,1.0,0.800000,0.0,0.333333,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
5,0,binary 0/1,860,race,Other,0.990909,1.0,0.985294,0.0,0.023256,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
6,0,binary 0/1,860,sex,Female,0.974576,1.0,0.960699,0.0,0.067164,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
7,0,binary 0/1,860,sex,Male,0.972414,1.0,0.947644,0.0,0.055096,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
8,0,binary 0/1,860,age_cat,25 - 45,0.966699,1.0,0.938849,0.0,0.068136,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45
9,0,binary 0/1,860,age_cat,Greater than 45,0.987654,1.0,0.981618,0.0,0.036232,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45


In [25]:
majority_bdf[['attribute_name', 'attribute_value'] +  audit.bias.list_disparities(majority_bdf)]

,attribute_name,attribute_value,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity
0,race,African-American,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,1.0,1.000000,1.0
1,race,Asian,0.008197,0.816940,1.058568,0.000000,NaN,0.000000,NaN,1.0,1.066015,1.0
2,race,Caucasian,0.549180,0.795824,0.995370,1.079049,NaN,0.704006,NaN,1.0,1.019540,1.0
3,race,Hispanic,0.110656,0.612705,0.980156,1.338820,NaN,0.576720,NaN,1.0,1.027943,1.0
4,race,Native American,0.006148,0.787763,0.705712,6.024691,NaN,3.229630,NaN,1.0,0.852812,1.0
5,race,Other,0.088115,0.718536,1.033950,0.420327,NaN,0.237473,NaN,1.0,1.050338,1.0
6,sex,Female,0.184573,0.756019,0.987229,1.219030,NaN,0.750655,NaN,1.0,1.013776,1.0
7,sex,Male,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,1.0,1.000000,1.0
8,age_cat,25 - 45,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,NaN,1.0,1.000000,1.0
9,age_cat,Greater than 45,0.276553,0.697187,1.034237,0.531756,NaN,0.300606,NaN,1.0,1.045554,1.0


In [29]:
min_metric_bdf = audit.bias.get_disparity_min_metric(xtab,original_df=result,
                                            check_significance=True)
min_metric_bdf.style

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3310: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3310: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3310: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\scipy\stats\_morestats.py:3025: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,label_value_significance,score_significance,fdr_disparity,fdr_ref_group_value,fdr_significance,fnr_disparity,fnr_ref_group_value,fnr_significance,for_disparity,for_ref_group_value,for_significance,fpr_disparity,fpr_ref_group_value,fpr_significance,npv_disparity,npv_ref_group_value,npv_significance,ppr_disparity,ppr_ref_group_value,ppr_significance,pprev_disparity,pprev_ref_group_value,pprev_significance,precision_disparity,precision_ref_group_value,precision_significance,tnr_disparity,tnr_ref_group_value,tnr_significance,tpr_disparity,tpr_ref_group_value,tpr_significance
0,0,binary 0/1,860,race,African-American,0.969900,1.000000,0.938073,0.000000,0.055328,0.061927,0.000000,1.000000,0.944672,488,409,0.567442,0.544036,27,0,409,461,461,436,897,1804,0.513935,False,False,10.000000,Asian,False,nan,African-American,None,nan,African-American,None,10.000000,Asian,False,1.000000,African-American,None,162.666667,Native American,False,1.632107,Hispanic,True,1.417008,Native American,False,1.172592,Native American,False,1.000000,African-American,None
1,0,binary 0/1,860,race,Asian,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,4,5,0.004651,0.444444,0,0,5,4,4,5,9,1804,0.444444,False,False,nan,Asian,None,nan,African-American,None,nan,African-American,None,nan,Asian,None,1.000000,African-American,None,1.333333,Native American,False,1.333333,Hispanic,False,1.500000,Native American,False,1.250000,Native American,False,1.000000,African-American,None
2,0,binary 0/1,860,race,Caucasian,0.974152,1.000000,0.956403,0.000000,0.059701,0.043597,0.000000,1.000000,0.940299,268,351,0.311628,0.432956,16,0,351,252,252,367,619,1804,0.407108,False,False,10.000000,Asian,False,nan,African-American,None,nan,African-American,None,10.000000,Asian,False,1.000000,African-American,None,89.333333,Native American,False,1.298869,Hispanic,True,1.410448,Native American,False,1.195504,Native American,False,1.000000,African-American,None
3,0,binary 0/1,860,race,Hispanic,0.975309,1.000000,0.964286,0.000000,0.074074,0.035714,0.000000,1.000000,0.925926,54,108,0.062791,0.333333,4,0,108,50,50,112,162,1804,0.308642,False,False,10.000000,Asian,False,nan,African-American,None,nan,African-American,None,10.000000,Asian,False,1.000000,African-American,None,18.000000,Native American,False,1.000000,Hispanic,False,1.388889,Native American,False,1.205357,Native American,False,1.000000,African-American,None
4,0,binary 0/1,860,race,Native American,0.857143,1.000000,0.800000,0.000000,0.333333,0.200000,0.000000,1.000000,0.666667,3,4,0.003488,0.428571,1,0,4,2,2,5,7,1804,0.285714,False,False,10.000000,Asian,False,nan,African-American,None,nan,African-American,None,10.000000,Asian,False,1.000000,African-American,None,1.000000,Native American,False,1.285714,Hispanic,False,1.000000,Native American,False,1.000000,Native American,False,1.000000,African-American,None
5,0,binary 0/1,860,race,Other,0.990909,1.000000,0.985294,0.000000,0.023256,0.014706,0.000000,1.000000,0.976744,43,67,0.050000,0.390909,1,0,67,42,42,68,110,1804,0.381818,False,False,10.000000,Asian,False,nan,African-American,None,nan,African-American,None,10.000000,Asian,False,1.000000,African-American,None,14.333333,Native American,False,1.172727,Hispanic,False,1.465116,Native American,False,1.231618,Native American,False,1.000000,African-American,None
6,0,binary 0/1,860,sex,Female,0.974576,1.000000,0.960699,0.000000,0.067164,0.039301,0.000000,1.000000,0.932836,134,220,0.155814,0.378531,9,0,220,125,125,229,354,1804,0.353107,False,False,1.219030,Male,False,nan,Female,None,nan,Female,None,1.000000,Female,False,1.000000,Female,None,1.000000,Female,False,1.000000,Female,False,1.000000,Female,False,1.013776,Male,False,1.000000,Female,None
7,0,binary 0/1,860,sex,Male,0.972414,1.000000,0.947644,0.000000,0.055096,0.052356,

In [31]:
f = Fairness()
fdf = f.get_group_value_fairness(bdf)
fdf

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,0,binary 0/1,860,race,African-American,0.969900,1.0,0.938073,0.0,0.055328,...,NaN,True,True,True,True,False,NaN,False,False,False
1,0,binary 0/1,860,race,Asian,1.000000,1.0,1.000000,0.0,0.000000,...,NaN,True,True,True,True,False,NaN,False,False,False
2,0,binary 0/1,860,race,Caucasian,0.974152,1.0,0.956403,0.0,0.059701,...,NaN,True,True,True,True,True,NaN,True,True,False
3,0,binary 0/1,860,race,Hispanic,0.975309,1.0,0.964286,0.0,0.074074,...,NaN,True,True,True,True,True,NaN,True,False,False
4,0,binary 0/1,860,race,Native American,0.857143,1.0,0.800000,0.0,0.333333,...,NaN,True,True,True,False,False,NaN,False,False,False
5,0,binary 0/1,860,race,Other,0.990909,1.0,0.985294,0.0,0.023256,...,NaN,True,True,True,True,False,NaN,False,False,False
6,0,binary 0/1,860,sex,Female,0.974576,1.0,0.960699,0.0,0.067164,...,NaN,True,True,True,True,False,NaN,False,False,False
7,0,binary 0/1,860,sex,Male,0.972414,1.0,0.947644,0.0,0.055096,...,NaN,True,True,True,True,True,NaN,True,True,False
8,0,binary 0/1,860,age_cat,25 - 45,0.966699,1.0,0.938849,0.0,0.068136,...,NaN,True,True,True,True,True,NaN,True,True,False
9,0,binary 0/1,860,age_cat,Greater than 45,0.987654,1.0,0.981618,0.0,0.036232,...,NaN,True,True,True,True,False,NaN,False,False,False


In [32]:
parity_detrminations = f.list_parities(fdf)
parity_detrminations

['Equalized Odds',
 'TypeII Parity',
 'TypeI Parity',
 'Unsupervised Fairness',
 'FPR Parity',
 'TPR Parity',
 'TNR Parity',
 'Supervised Fairness',
 'FOR Parity',
 'Impact Parity',
 'Precision Parity',
 'FNR Parity',
 'Statistical Parity',
 'NPV Parity',
 'FDR Parity']

In [34]:
fdf[['attribute_name', 'attribute_value'] + audit.group.list_absolute_metrics(result) + audit.bias.list_disparities(fdf) + parity_detrminations].style

,attribute_name,attribute_value,fdr_disparity,fnr_disparity,for_disparity,fpr_disparity,npv_disparity,ppr_disparity,pprev_disparity,precision_disparity,tnr_disparity,tpr_disparity,Equalized Odds,TypeII Parity,TypeI Parity,Unsupervised Fairness,FPR Parity,TPR Parity,TNR Parity,Supervised Fairness,FOR Parity,Impact Parity,Precision Parity,FNR Parity,Statistical Parity,NPV Parity,FDR Parity
0,race,African-American,0.926742,nan,nan,1.420442,1.000000,1.820896,1.256560,1.004651,0.980835,1.000000,False,nan,False,False,False,True,True,False,nan,False,True,nan,False,True,True
1,race,Asian,0.000000,nan,nan,0.000000,1.000000,0.014925,1.026534,1.063492,1.045584,1.000000,False,nan,False,False,False,True,True,False,nan,True,True,nan,False,True,False
2,race,Caucasian,1.000000,nan,nan,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,True,nan,True,True,True,True,True,False,nan,True,True,nan,True,True,True
3,race,Hispanic,1.240741,nan,nan,0.819196,1.000000,0.201493,0.769900,0.984715,1.008242,1.000000,True,nan,True,False,True,True,True,False,nan,False,True,nan,False,True,True
4,race,Native American,5.583333,nan,nan,4.587500,1.000000,0.011194,0.989872,0.708995,0.836467,1.000000,False,nan,False,False,False,True,True,False,nan,True,False,nan,False,True,False
5,race,Other,0.389535,nan,nan,0.337316,1.000000,0.160448,0.902883,1.038760,1.030208,1.000000,False,nan,False,False,False,True,True,False,nan,True,True,nan,False,True,False
6,sex,Female,1.219030,nan,nan,0.750655,1.000000,0.184573,0.756019,0.987229,1.013776,1.000000,False,nan,False,False,False,True,True,False,nan,False,True,nan,False,True,True
7,sex,Male,1.000000,nan,nan,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,True,nan,True,True,True,True,True,False,nan,True,True,nan,True,True,True
8,age_cat,25 - 45,1.000000,nan,nan,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,True,nan,True,True,True,True,True,False,nan,True,True,nan,True,True,True
9,age_cat,Greater than 45,0.531756,nan,nan,0.300606,1.000000,0.276553,0.697187,1.034237,1.045554,1.000000,False,nan,False,False,False,True,True,False,nan,False,True,nan,False,True,False


In [35]:
gaf = f.get_group_attribute_fairness(fdf)
gaf

,model_id,score_threshold,attribute_name,Statistical Parity,Impact Parity,FDR Parity,FPR Parity,FOR Parity,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,0,binary 0/1,age_cat,False,False,False,False,NaN,NaN,True,True,True,True,False,NaN,False,False,False
1,0,binary 0/1,race,False,False,False,False,NaN,NaN,True,True,True,False,False,NaN,False,False,False
2,0,binary 0/1,sex,False,False,True,False,NaN,NaN,True,True,True,True,False,NaN,False,False,False
